<a href="https://colab.research.google.com/github/FerronattoJose/Bovine_Milk_Microbiome/blob/main/Pipeline_16S_JAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## QIIME 2

## Importing Data into QIIME 2

In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input- .gz \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

This command imports raw paired-end sequencing data into QIIME 2.

## Removing Primers with Cutadapt

In [ ]:
qiime cutadapt trim-paired \
--i-demultiplexed-sequences demux.qza \
--p-front-f TCGTCGGCAGCGTCAGATGTGTATAAGAGACAGCCTACGGGNGGCWGCAG \
--p-front-r GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAGGACTACHVGGGTATCTAATCC \
--p-discard-untrimmed \
--verbose \
--o-trimmed-sequences trimmed_remove_primers.qza

This command removes primers from the sequences using Cutadapt.

## Extracting Trimmed Reads

In [ ]:
qiime tools extract --input-path trimmed_remove_primers.qza  --output-path extracted_reads

This command extracts the processed reads from the QIIME 2 artifact for inspection.

## Summarizing the demultiplexed sequences

qiime demux summarize --i-data demux.qza --o-visualization demux.qzv


Generates a summary of the demultiplexed sequences, including quality plots.

## Denoising with DADA2 (single-end reads)

qiime dada2 denoise-single \
--i-demultiplexed-seqs demux.qza \
--p-trim-left 10 \
--p-trunc-len 200 \
--o-representative-sequences rep-seqs-dada2.qza \
--o-table table-dada2.qza \
--o-denoising-stats stats-dada2.qza \
--p-n-threads 0


Performs quality filtering, error correction, merging (if needed), and chimera removal using the DADA2 algorithm.

## Summarizing the feature table

In [ ]:
qiime feature-table summarize \
--i-table table.qza \
--o-visualization table.qzv


Creates a summary visualization of the ASV table, showing sequencing depth per sample.

## Viewing representative sequences

In [ ]:
qiime feature-table tabulate-seqs --i-data rep-seqs.qza --o-visualization rep-seqs.qzv


Generates a table displaying the representative sequences (ASVs) with sequence IDs.

## Viewing DADA2 denoising statistics

In [ ]:
qiime metadata tabulate --m-input-file stats-dada2.qza --o-visualization stats-dada2.qzv


Creates a visualization summarizing DADA2 processing statistics (e.g., how many sequences were retained at each step). P.s. all qzv files were viewed at https://view.qiime2.org/

## Summarizing the feature table with metadata

In [ ]:
qiime feature-table summarize --i-table table.qza --o-visualization table.qzv --m-sample-metadata-file metadata.txt


Creates a summary visualization of the feature table while incorporating sample metadata.

## Building a phylogenetic tree

In [ ]:
qiime phylogeny align-to-tree-mafft-fasttree \
--i-sequences rep-seqs.qza \
--o-alignment aligned-rep-seqs.qza \
--o-masked-alignment masked-aligned-rep-seqs.qza \
--o-tree unrooted-tree.qza \
--o-rooted-tree rooted-tree.qza


Constructs a phylogenetic tree of representative sequences for phylogeny-based analyses (e.g., UniFrac).

## Taxonomic classification

In [ ]:
qiime feature-classifier classify-sklearn \
--i-classifier silva-138.2-ssu-nr99-classifier \
--i-reads rep-seqs.qza \
--o-classification taxonomy.qza


Assigns taxonomy to representative sequences using a trained classifier.

## Transposing the feature table

In [ ]:
qiime feature-table transpose \
  --i-table table.qza \
  --o-transposed-feature-table transposed-table.qza


This command transposes the feature table, swapping features (ASVs/OTUs) and samples.

## Merging metadata tables

In [ ]:
qiime metadata tabulate \
  --m-input-file transposed-table.qza \
  --m-input-file taxonomy.qza \
  --m-input-file rep-seqs.qza \
  --o-visualization merged.qzv


This command merges multiple metadata sources into a single visual file.

## Viewing taxonomic classifications

In [ ]:
qiime metadata tabulate --m-input-file taxonomy.qza --o-visualization taxonomy.qzv


Creates a visualization of the taxonomic classifications for each ASV.

## Filtering features based on taxonomy

In [ ]:
qiime feature-table filter-features \
  --i-table table.qza \
  --m-metadata-file taxonomy.qza \
  --o-filtered-table id-filtered-table.qza


Filters the feature table to keep only features that have taxonomic assignments.

## Collapsing the feature table to genus level

In [ ]:
qiime taxa collapse \
  --i-table id-filtered-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-level 6 \
  --o-collapsed-table genus_table.qza


Collapses ASVs to the genus level (L6) by summing their abundances.

## Creating a taxonomy bar plot

In [ ]:
qiime taxa barplot \
  --i-table id-filtered-table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file metadata.txt \
  --o-visualization taxa-bar-plots.qzv


Generates a taxonomic bar plot showing relative abundances of different taxa per sample.

## Exporting feature tables and taxonomy

In [ ]:
qiime tools export --input-path genus_table-l6.qza --output-path exported-feature-table
qiime tools export --input-path taxonomy.qza --output-path exported-feature-table


Exports the genus-level feature table and taxonomy assignments as external files.

## Filtering out unwanted taxa

In [ ]:
qiime taxa filter-table \
  --i-table id-filtered-table.qza \
  --i-taxonomy taxonomy.qza \
  --p-exclude Unassigned,d__Archaea,p__Chloroflexi,p__Cyanobacteria,g__Mitochondria \
  --o-filtered-table filtered_table.qza


Removes specific unwanted taxa from the feature table.

## Converting BIOM to TSV format

In [ ]:
biom convert -i feature-table.biom -o table.from_biom.txt --to-tsv


Converts the BIOM (Binary OTU Table) format into a human-readable TSV (tab-separated values) format.

# Rstudio

## Reading Data from QIIME 2

In [ ]:
library(phyloseq)   # For microbiome analysis
library(qiime2R)    # To read QIIME 2 files (.qza)
library(dplyr)      # For data manipulation
library(tidyverse)  # For general data processing

#Load your data
SVs <- read_qza("table.qza")
metadata <- read_q2metadata("metadata.tsv")
head(metadata) # show top lines of metadata

taxonomy <- read_qza("taxonomy.qza")
head(taxonomy$data)



## Creating a Phyloseq Object

In [ ]:
#Combines the feature table, tree, taxonomy, and metadata into a phyloseq object.
physeq <- qza_to_phyloseq(
  features="table.qza",
  tree="rooted-tree.qza",
  taxonomy="taxonomy.qza",
  metadata = "metadata.tsv"
)
physeq


#save the phyloseq object
saveRDS(physeq, "physeq")


## Extracting Unique Genera

In [ ]:
#Extracts and lists the unique genus-level classifications.
unique(tax_table(physeq)[,"Genus"])

#Assigns unique ASV names (ASV1, ASV2, etc.) and adds the original ASV sequence as a new column in the taxonomy table.
taxa_names(physeq) <- paste0("ASV", seq(ntaxa(physeq)))
tax_table(physeq) <- cbind(tax_table(physeq),
                           rownames(tax_table(physeq)))
head(taxa_names(physeq))


#Renames the columns of the taxonomy table for clarity.
colnames(tax_table(physeq)) <- c("Kingdom", "Phylum", "Class", "Order",
                                 "Family", "Genus", "ASV_SEQ", "ASV_ID")



## Filtering Unwanted Taxa

In [ ]:
#Removes Eukaryota sequences.
EU1 <- subset_taxa(physeq, Kingdom == "d__Eukaryota")
EU1 <- as(tax_table(EU1), "matrix")
EU1 <- EU1[, 8]
EU1df <- as.factor(EU1)
goodTaxa <- setdiff(taxa_names(physeq), EU1df)
phyloseq_no_euk <- prune_taxa(goodTaxa, physeq)

#Removes ASVs that are unassigned at the kingdom level.
NA1 <- subset_taxa(physeq, Kingdom == "Unassigned")
NA1 <- as(tax_table(NA1), "matrix")
NA1 <- NA1[, 8]
NA1df <- as.factor(NA1)
goodTaxa <- setdiff(taxa_names(physeq), NA1df)
phyloseq_no_NA <- prune_taxa(goodTaxa, physeq)

#Removes Archaea sequences.
NA2 <- subset_taxa(phyloseq_no_NA, Kingdom == "d__Archaea")
NA2 <- as(tax_table(NA2), "matrix")
NA2 <- NA2[, 8]
NA2df <- as.factor(NA2)
goodTaxa <- setdiff(taxa_names(phyloseq_no_NA), NA2df)
phyloseq_no_Arch <- prune_taxa(goodTaxa, phyloseq_no_NA)

#Removes Cyanobacteria sequences.
NA3 <- subset_taxa(phyloseq_no_Arch, Phylum == "Cyanobacteriota")
NA3 <- as(tax_table(NA3), "matrix")
NA3 <- NA3[, 8]
NA3df <- as.factor(NA3)
goodTaxa <- setdiff(taxa_names(phyloseq_no_Arch), NA3df)
phyloseq_no_Cyano <- prune_taxa(goodTaxa, phyloseq_no_Arch)

#Removes Mitochondria sequences.
NA4 <- subset_taxa(phyloseq_no_Cyano, Family == "Mitochondria")
NA4 <- as(tax_table(NA4), "matrix")
NA4 <- NA4[, 8]
NA4df <- as.factor(NA4)
goodTaxa <- setdiff(taxa_names(phyloseq_no_Cyano), NA4df)
phyloseq_no_Unidentified <- prune_taxa(goodTaxa, phyloseq_no_Cyano)


#Filters out taxa that do not have a Phylum classification and removes unidentified taxa (e.g., those labeled as NA at the Phylum level).
phyloseq <- subset_taxa(phyloseq_no_Unidentified, !is.na(Phylum))
phyloseq

#Replaces "d__Bacteria" with a cleaner name: "Bacteria" and ensures consistency in taxonomy labels.
tax_table(phyloseq)[tax_table(phyloseq) == "d__Bacteria"] <- "Bacteria"



## Decontam

In [ ]:
#https://benjjneb.github.io/decontam/vignettes/decontam_intro.html

#Identifying Contaminants via FREQUENCY Method
#Base on DNA concentration


# Assigning phyloseq object
ps5 <- phyloseq

# Checking sample metadata
head(sample_data(ps5))

#Computes library sizes (total reads per sample) and sorts them and plots a scatter plot to visualize sequencing depth by sample type.
df <- as.data.frame(sample_data(ps5))  # Convert sample metadata to a data frame
df$LibrarySize <- sample_sums(ps5)     # Compute library size (total reads per sample)
df <- df[order(df$LibrarySize),]       # Order samples by library size
df$Index <- seq(nrow(df))              # Create an index for plotting

# Scatter plot of library size
ggplot(data=df, aes(x=Index, y=LibrarySize, color=Sample_or_control)) +
  geom_point()

#Uses the FREQUENCY method to detect contaminants, based on DNA concentration (quant_reading). Contaminants are identified when their abundance correlates with concentration.
contam.freq5 <- isContaminant(ps5, conc="quant_reading", method="frequency", threshold=0.5)

temp_contam.freq5 <- contam.freq5
rownames(temp_contam.freq5) <- paste0("SV",seq(nrow(temp_contam.freq5)))  # Shorten row names
print(head(temp_contam.freq5))  # Show first 10 rows

#Randomly selects a contaminant and plots its abundance vs. concentration.
set.seed(100)
plot_frequency(ps5, taxa_names(ps5)[sample(which(contam.freq5$contaminant),1)], conc="quant_reading")


#Identifying Contaminants via PREVALENCE Method
#Base on NEG1 (DNA extraction control)

ps5.1 <- phyloseq

# Compute library size
df <- as.data.frame(sample_data(ps5.1))
df$LibrarySize <- sample_sums(ps5.1)
df <- df[order(df$LibrarySize),]
df$Index <- seq(nrow(df))

# Scatter plot
ggplot(data=df, aes(x=Index, y=LibrarySize, color=Sample_or_control)) +
  geom_point()

# Define negative control
sample_data(ps5.1)$is.neg <- sample_data(ps5.1)$Sample_or_control == "NEG1"

# Identify contaminants
contamdf.prev <- isContaminant(ps5.1, method="prevalence", neg="is.neg")
table(contamdf.prev$contaminant)
head(which(contamdf.prev$contaminant))

# Apply threshold 0.5
contamdf.prev05 <- isContaminant(ps5.1, method="prevalence", neg="is.neg", threshold=0.5)
table(contamdf.prev05$contaminant)

#Visualizing Prevalence in "NEG1" vs. True Samples.
ps5.1.pa <- transform_sample_counts(ps5.1, function(abund) 1*(abund>0))
ps5.1.pa.neg <- prune_samples(sample_data(ps5.1.pa)$Sample_or_control == "NEG1", ps5.1.pa)
ps5.1.pa.pos <- prune_samples(sample_data(ps5.1.pa)$Sample_or_control == "True_Sample", ps5.1.pa)

df.pa <- data.frame(pa.pos=taxa_sums(ps5.1.pa.pos), pa.neg=taxa_sums(ps5.1.pa.neg),
                    contaminant=contamdf.prev$contaminant)

ggplot(data=df.pa, aes(x=pa.neg, y=pa.pos, color=contaminant)) +
  geom_point() +
  xlab("Prevalence (NEG1)") + ylab("Prevalence (True_Sample)")

#Repeat
#Base on NEG2 (collection control)

ps5.2 <- phyloseq
sample_data(ps5.2)$is.neg <- sample_data(ps5.2)$Sample_or_control == "NEG2"
contamdf.prev05.2 <- isContaminant(ps5.2, method="prevalence", neg="is.neg", threshold=0.5)
table(contamdf.prev05.2$contaminant)

ps5.2.pa <- transform_sample_counts(ps5.2, function(abund) 1*(abund>0))
ps5.2.pa.neg <- prune_samples(sample_data(ps5.2.pa)$Sample_or_control == "NEG2", ps5.2.pa)
ps5.2.pa.pos <- prune_samples(sample_data(ps5.2.pa)$Sample_or_control == "True_Sample", ps5.2.pa)

df.pa <- data.frame(pa.pos=taxa_sums(ps5.2.pa.pos), pa.neg=taxa_sums(ps5.2.pa.neg),
                    contaminant=contamdf.prev$contaminant)

ggplot(data=df.pa, aes(x=pa.neg, y=pa.pos, color=contaminant)) +
  geom_point() +
  xlab("Prevalence (NEG2)") + ylab("Prevalence (True_Sample)")


#Exporting Contaminant Results
#Saves contamination results in both Excel (.xlsx) and text (.txt) formats.

library("writexl")

write_xlsx(contamdf.prev05,"decontamination05_ASV.xlsx", col_names = TRUE, format_headers = TRUE)
write_xlsx(contamdf.prev05,"decontamination05_ASV.xlsx", col_names = TRUE, format_headers = TRUE)
write_xlsx(contam.freq5,"decontamination05_ASV.freq.xlsx", col_names = TRUE, format_headers = TRUE)

write.table(contamdf.prev05, "decontamination_05_.txt", sep="\t", quote = FALSE, col.names=NA)
write.table(contamdf.prev05, "decontamination_05_.txt", sep="\t", quote = FALSE, col.names=NA)
write.table(contam.freq5, "decontamination.freq_05_.txt", sep="\t", quote = FALSE, col.names=NA)

#I did the same with 5.1 and 5.2

#Saving Processed Taxonomy & ASV Tables. Saves taxonomy and ASV tables after decontamination.

unique(tax_table(phyloseq_no_after5)[,"ASV_ID"])

write.table(tax_table(phyloseq_no_after5), "trim_tax_table_ASVID.txt", sep="\t", quote = FALSE, col.names=NA)
write.table(t(otu_table(phyloseq_no_after5)), "trim_seq_table_ASVID.txt", sep="\t", quote = FALSE, col.names=NA)


#Remove controls for phyloseq
phyloseq_before_withoutneg = subset_samples(phyloseq, Sample_or_control %in% c("True_Sample"))
phyloseq_before_withoutneg


#Remove Contaminants for ps5
ps5.cleaned05 <- prune_taxa(!contamdf.prev05$contaminant, ps5)  # Remove contaminants from ps5
phyloseq_5_withoutneg = subset_samples(ps5.cleaned05, Sample_or_control %in% c("True_Sample")) #Remove control samples
phyloseq_5_withoutneg

#I applied the same for 0.6 and 0.8

## Relative abundance

In [ ]:
#Remove singletons
ps_rel <- filter_taxa(ps, function(x) sum(x > 0) > (0.2*length(x)), TRUE)

#Summarizes the phyloseq object ps (e.g., number of taxa, samples, reads).
microbiome::summarize_phyloseq(ps)

ps2 <- ps

#ps2.rel converts the data to relative abundance (compositional transformation).
ps2.rel <- microbiome::transform(ps2, "compositional")


#Aggregate Rare Taxa
ps2.ASV.rel <- aggregate_rare(ps2.rel, level = "ASV_ID", detection = 0.01, prevalence = 0.1)
ps2.phy.rel <- aggregate_rare(ps2.rel, level = "Phylum", detection = 0.05, prevalence = 0.1)
ps2.fam.rel <- aggregate_rare(ps2.rel, level = "Family", detection = 0.05, prevalence = 0.1)
ps2.gen.rel <- aggregate_rare(ps2.rel, level = "Genus", detection = 0.05, prevalence = 0.1)

#Define the color palette (I also have this for Family, Phylum and ASV level)
colorsge <- c("Bacillus" = "#2A363BFF",
              "Cutibacterium" = "#749E89FF",
              "Pedomicrobium" = "#FFB6C1",
              "Other" = "#FECEA8FF",
              "Pseudomonas" = "#FF847CFF",
              "Staphylococcus" = "#9F6E71FF",
              "Unknown" = "#41507BFF",
              "Sediminibacterium" = "#99B898FF",
              "Acinetobacter" = "#ABCCBEFF",
              "Corynebacterium" = "#E3CACFFF")


#Creates a bar plot showing relative abundance per ASV.
p_avg_ASV <- plot_composition(ps2.ASV.rel, average_by = "Group") +
  guides(fill = guide_legend(ncol = 1)) +
  labs(x = "Group",
       y = "Relative Abundance",
       title = "Before Decontam with PMA - SILVA 138.2") +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1),
        axis.ticks.x = element_blank(),
        strip.background = element_rect(fill = "white"))

print(p_avg_ASV)


#Plots phylum-level relative abundance averaged by group.
p_avg_phy <- plot_composition(ps2.phy.rel, average_by = "Group") +
  guides(fill = guide_legend(ncol = 1)) +
  labs(x = "Group",
       y = "Relative abundance",
       title = "Before Decontam - SILVA_138.2") +
  scale_fill_manual("Phylum", values = colorsph) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
p_avg_phy


#Plots family-level relative abundance averaged by group.
p_avg_fam <- plot_composition(ps2.fam.rel,
                              average_by = "Group") +
  guides(fill = guide_legend(ncol = 1)) +
  labs(x = "Group",
       y = "Relative abundance",
       title = "Before Decontam - SILVA_138.2") +
  scale_fill_manual("Family", values = colorsfa) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
p_avg_fam



#Plots genus-level relative abundance averaged by group.
p_avg_gen <- plot_composition(ps2.gen.rel,
                              average_by = "Group") +
  guides(fill = guide_legend(ncol = 1)) +
  labs(x = "Group",
       y = "Relative abundance",
       title = "Before Decontam - SILVA_138.2") +
  scale_fill_manual("Genus", values = colorsge) +
  theme_bw() +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
  guides(fill = guide_legend(ncol = 1))
p_avg_gen


# I did the same with 0.5, 0.6 and 0.8

## Alfa diversity

In [ ]:
# Load libraries
library(ggplot2)      # For plotting
library(dplyr)        # Data manipulation
library(tidyr)        # Data transformation
library(phyloseq)     # Microbiome data analysis
library(microbiome)   # Diversity and evenness calculations
library(ggpubr)       # Statistical comparisons (stat_compare_means)



#Calculates Chao1, Shannon, and Observed alpha diversity indices for ps
tab <- microbiome::alpha(ps, index = c("Chao1", "Shannon", "Observed"))

#Extracts metadata and adds the computed diversity values to the ps.meta dataframe.
ps.meta <- meta(ps)
ps.meta$Shannon <- tab$diversity_shannon
ps.meta$chao1 <- tab$chao1
ps.meta$Observed <- tab$observed

#evenness index
evenness <- microbiome::evenness(ps)

#Converts the evenness results into a dataframe.
evenness_df <- data.frame(SampleID = rownames(evenness), Evenness = evenness$pielou)

#Merges evenness values with sample metadata.
metadata <- data.frame(sample_data(ps))
evenness_df <- merge(evenness_df, metadata, by.x = "SampleID", by.y = "row.names")


#Creates a boxplot of the EVENNESS index, grouped by Group (with and without PMA).
ggplot(evenness_df, aes(x = Group, y = Evenness, fill = Group)) +
  geom_boxplot(alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  scale_fill_manual(values = c("without PMA" = "steelblue", "with PMA" = "coral")) +
  labs(title = "Evenness Index Distribution", y = "Evenness (Pielou's Index)") +
  theme_minimal()


#Creates a dataframe containing the OBSERVED ASV counts and merges it with metadata.
diversity_metrics <- data.frame(SampleID = rownames(tab),
                                Observed_ASVs = tab$observed)
diversity_metrics <- merge(diversity_metrics, ps.meta, by.x = "SampleID", by.y = "row.names")
diversity_metrics$Group <- as.factor(diversity_metrics$Group)

#Creates a boxplot of the distribution of observed ASVs per group.
ggplot(diversity_metrics, aes(x = Group, y = Observed_ASVs, fill = Group)) +
  geom_boxplot(alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.5, color = "black") +
  scale_fill_manual(values = c("with PMA" = "coral", "without PMA" = "steelblue")) +
  labs(title = "Observed ASVs Distribution by Group", y = "Observed ASVs", x = "Group") +
  theme_minimal()


#Shannon Diversity
p.shannon <- boxplot_alpha(ps1_filtered,
                           index = "shannon",
                           x_var = "Group",
                           na.rm = TRUE)

p.shannon <- p.shannon + theme_minimal() +
  labs(x="Group", y="Shannon diversity") +
  theme(axis.text = element_text(size=12),
        axis.title = element_text(size=16),
        legend.text = element_text(size=12),
        legend.title = element_text(size=16))

#Computes statistical comparisons between groups and adds them to the plot.
stat <- levels(ps1.meta$group)
stat.pairs <- combn(seq_along(stat), 2, simplify = FALSE, FUN = function(i)stat[i])
p.shannon <- p.shannon + stat_compare_means(comparisons = stat.pairs, size = 7)
print(p.shannon)


#Chao1 Diversity
p.chao <- boxplot_alpha(ps,
                        index = "chao1",
                        x_var = "Group",
                        na.rm = TRUE)

p.chao <- p.chao + theme_minimal() +
  labs(x="Group", y="Chao1 diversity") +
  theme(axis.text = element_text(size=12),
        axis.title = element_text(size=16),
        legend.text = element_text(size=12),
        legend.title = element_text(size=16))

#Adds statistical comparisons to the Chao1 diversity plot.
p.chao <- p.chao + stat_compare_means(comparisons = stat.pairs)
print(p.chao)

#I did the same with differents phyloseq's (0.5, 0.6 and 0.8) and differents variables (SCC, microbiological, etc)

## Beta diversity

In [ ]:
library(phyloseq)       # For microbiome data analysis
library(ggplot2)        # For data visualization
library(vegan)          # For PERMANOVA (adonis2)
library(ggrepel)        # For improved text labeling in plots
library(pairwiseAdonis) # For pairwise PERMANOVA analysis


#This is an example how I subset the group (with and without PMA). This filters the phyloseq object, keeping only samples where Group == "with PMA".
ps2.rel_PMA <- subset_samples(ps, Group == "with PMA")

#Unweighted UniFrac Distance & PCoA Ordination

#Calculates Unweighted UniFrac distances for ps2.rel_PMA.
wunifrac_dist = phyloseq::distance(ps2.rel_PMA, method="unifrac", weighted=FALSE)

#Performs Principal Coordinates Analysis (PCoA) on the distance matrix.
ordination10 = ordinate(ps2.rel_PMA, method="PCoA", distance=wunifrac_dist)


#Plots the PCoA ordination using Unweighted UniFrac distances.
plot_ordination(ps2.rel_PMA, ordination10, color = "SCC_1") +
  theme(
    aspect.ratio = 1,
    axis.text = element_text(size = 18),
    axis.title = element_text(size = 24),
    legend.text = element_text(size = 18),
    legend.title = element_blank()
  ) +
  ggtitle("SCC_1_with_PMA_UW_Unifrac") +
  stat_ellipse(geom = "polygon", type = "norm", alpha = 0.1, fill = NA) +  # Removes ellipse fill
  geom_point(size = 1.5) +
  geom_text_repel(aes(label = Quarters), size = 2, max.overlaps = 130) +
  theme_classic()


# PERMANOVA (Adonis) to Test for Group Differences - Compares microbial composition across SCC (somatic cell count) levels.
adonis2(wunifrac_dist ~ sample_data(ps2.rel_PMA)$SCC_1)


#Pairwise Comparison of Samples in Metadata vs. Distance Matrix

metadata_samples <- rownames(metadata)
dist_samples <- labels(wunifrac_dist)

#Identifies mismatches between the UniFrac distance matrix and metadata.
extra_in_dist <- setdiff(dist_samples, metadata_samples)
extra_in_metadata <- setdiff(metadata_samples, dist_samples)

print(extra_in_dist)
print(extra_in_metadata)

#Filters dist_unifrac to include only common samples and Ensures matrix size matches metadata sample count.
common_samples <- intersect(metadata_samples, dist_samples)
dist_unifrac <- as.dist(as.matrix(dist_unifrac)[common_samples, common_samples])
print(attr(dist_unifrac, "Size"))


#Pairwise PERMANOVA
pairwise_results <- pairwise.adonis(dist_unifrac, factors = metadata$SCC_1)
print(pairwise_results)



#Weighted UniFrac Distance & PCoA Ordination
#Calculates Weighted UniFrac distances (accounts for relative abundance differences).
wunifrac_dist = phyloseq::distance(ps2.rel_PMA, method="unifrac", weighted=TRUE)

#Performs PCoA on the weighted distance matrix.
ordination10.1 = ordinate(ps2.rel_PMA, method="PCoA", distance=wunifrac_dist)

#PCoA Plot with Weighted UniFrac
plot_ordination(ps2.rel_PMA, ordination10.1, color = "SCC_1") +
  theme(
    aspect.ratio = 1,
    axis.text = element_text(size = 18),
    axis.title = element_text(size = 24),
    legend.text = element_text(size = 18),
    legend.title = element_blank()
  ) +
  ggtitle("SCC_1_with_PMA_W_Unifrac") +
  stat_ellipse(geom = "polygon", type = "norm", alpha = 0.1, fill = NA) +
  geom_point(size = 1.5) +
  geom_text_repel(aes(label = Quarters), size = 2, max.overlaps = 130) +
  theme_classic()


# PERMANOVA (Adonis) to Test for Group Differences - Compares microbial composition across SCC (somatic cell count) levels.
adonis2(wunifrac_dist ~ sample_data(ps2.rel_PMA)$SCC_1)


#Pairwise Comparison of Samples in Metadata vs. Distance Matrix

metadata_samples <- rownames(metadata)
dist_samples <- labels(wunifrac_dist)

#Identifies mismatches between the UniFrac distance matrix and metadata.
extra_in_dist <- setdiff(dist_samples, metadata_samples)
extra_in_metadata <- setdiff(metadata_samples, dist_samples)

print(extra_in_dist)
print(extra_in_metadata)

#Filters dist_unifrac to include only common samples and Ensures matrix size matches metadata sample count.
common_samples <- intersect(metadata_samples, dist_samples)
dist_unifrac <- as.dist(as.matrix(dist_unifrac)[common_samples, common_samples])
print(attr(dist_unifrac, "Size"))


#Pairwise PERMANOVA
pairwise_results <- pairwise.adonis(dist_unifrac, factors = metadata$SCC_1)
print(pairwise_results)


# I used the same code for differents phyloseq's (0.5, 0.6 and 0.8) and differents variables (SCC, Microbiological, etc).

## Differential abundance analysis

In [ ]:
# Load required libraries
library ("DESeq2")  # For differential expression analysis
library ("ggplot2")  # For plotting
library ("ggrepel")  # For adding text labels in plots
library ("phyloseq")  # For phyloseq object handling
library ("dplyr")  # For data manipulation


#phyloseq_to_deseq2: Converts a phyloseq object into a DESeq2 object for differential expression analysis. Here, SCC_1 is the variable to model.
ps_deseq <- phyloseq_to_deseq2(ps2.rel_w_PMA, ~ SCC_1)

# Estimate Size Factors
#apply(counts(ps_deseq), 1, ...): For each gene (ASV), computes the geometric mean of the counts to account for zero values.
geoMeans <- apply(counts(ps_deseq), 1, function(x) if (all(x == 0)) NA else exp(mean(log(x[x > 0]))))

#estimateSizeFactors: Estimates the scaling factors used to normalize count data.
dds <- estimateSizeFactors(ps_deseq, geoMeans=geoMeans)

#DESeq: Runs the differential expression analysis.
dds <- DESeq(dds)

#results(dds): Extracts the results from the DESeq2 analysis, which contains fold changes and p-values.
res <- results(dds)

#plotMA: Creates an MA plot (mean vs. fold-change), which is used to visualize the results of differential expression analysis.
plotMA(res, main="DESeq2 MA Plot", ylim=c(-5, 5))

res_df <- as.data.frame(res)

#res_df$significance: Labels significant results based on a threshold of p-value < 0.05 and absolute fold-change > 1.
res_df$significance <- ifelse(res_df$pvalue < 0.05 & abs(res_df$log2FoldChange) > 1, "Significant", "Not Significant")

#res_df$GroupSide: Categorizes results based on positive or negative log2 fold change (High/Low).
res_df$GroupSide <- ifelse(res_df$log2FoldChange > 0, "High", "Low")


#Visualize the result
ggplot(res_df, aes(x = log2FoldChange, y = -log10(pvalue), color = GroupSide)) +
  geom_point(alpha = 0.6, size = 5) +
  scale_color_manual(values = color_palette) +
  theme_minimal() +
  xlab("Log2 Fold Change") +
  ylab("-Log10 p-value") +
  ggtitle("Volcano Plot of DESeq2 SCC_1 - WPMA before Decontam") +
  theme(legend.title = element_blank()) +
  theme(legend.position = "top") +
  geom_text_repel(aes(label = label), size = 3, box.padding = 0.3, point.padding = 0.3,color = "black", show.legend = FALSE)



#Second option for graph
#Lollipop graph

#res_df_filtered: Filters the results for genera with significant fold changes (|log2FoldChange| > 0.5) and p-values < 0.05.
res_df_filtered <- res_df %>%
  filter(abs(log2FoldChange) > 0.5, pvalue < 0.05, !is.na(Genus))

#Visualize the result
ggplot(res_df_filtered, aes(x = log2FoldChange, y = Genus)) +
  geom_segment(aes(xend = 0, yend = Genus), colour = "grey50") +
  geom_point(size = 3, aes(colour = GroupSide)) +
  scale_colour_manual(values = c("High" = "green", "Low" = "purple"), limits = c("High", "Low")) +
  theme_bw() +
  theme(panel.grid.major.y = element_blank(), panel.grid.major.x = element_blank()) +
  facet_grid(scales = "free_y", space = "free_y") +
  labs(x = "Log2 Fold Change", y = "Genus") +
  ggtitle("Lollipop Plot with Horizontal Segments by Group - SCC_1 - WPMA")


# I did the samewith differents phyloseq's (0.5, 0.6 and 0.8) and differents variables (SCC, Microbiological, etc).





